In [49]:
!pip install -q -U kaggle_environments

^C
Note: you may need to restart the kernel to use updated packages.


In [50]:
import numpy as np
import pandas as pd
import random


import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

In [194]:
%%writefile rock.py
#always "rock"
def rock(observation, configuration):
    return 0


Overwriting rock.py


In [195]:
%%writefile paper.py
#always "paper"
def paper(observation, configuration):
    return 1


Overwriting paper.py


In [196]:
%%writefile scissors.py
#always "scissors"
def scissors(observation, configuration):
    return 2

Overwriting scissors.py


In [198]:
%%writefile rand.py
#random
import random

def rand(observation, configuration):
    return random.randrange(0, 3)

Overwriting rand.py


In [199]:
%%writefile random_r_or_p.py
#random "rock" or "paper"
import random

def rand(observation, configuration):
    return random.randrange(0, 2)

Writing random_r_or_p.py


In [200]:
%%writefile random_s_or_p.py
#random "scissors" or "paper"
import random

def rand(observation, configuration):
    return random.randrange(1, 3)

Writing random_s_or_p.py


In [202]:
%%writefile random_s_or_r.py
#random "scissors" or "rock"
import random

def rand(observation, configuration):
    return random.choice([0, 2])

Writing random_s_or_r.py


In [165]:
%%writefile copy_opponent.py
import random

def copy_opponent(observation, configuration):
    if observation.step > 0:       
        return observation.lastOpponentAction
    else:  
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


In [230]:
%%writefile reactionary.py
import random
from utils import get_score

last_react_action = None


def reactionary(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

Overwriting reactionary.py


In [185]:
%%writefile counter_reactionary.py

import random


from kaggle_environments import make, evaluate


last_counter_action = None


def counter_reactionary(observation, configuration):
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action

Overwriting counter_reactionary.py


In [192]:
%%writefile statistical.py
import random
action_histogram = {}


def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Writing statistical.py


In [231]:
evaluate(
    "rps", 
    ["rock.py", "reactionary.py"], 
    configuration={"episodeSteps": 100}
)

[[1, None]]

In [184]:
evaluate(
    "rps", 
    ["counter_reactionary.py", "counter_reactionary.py"], 
    configuration={"episodeSteps": 100}
)

[[None, None]]

In [223]:
list_names = [
    "rock", 
    "paper", 
    "scissors",
    "rand",  
    "random_r_or_p", 
    "random_s_or_p", 
    "random_s_or_r", 
    "copy_opponent",
    "statistical", 
]
# "counter_reactionary","reactionary"
list_agents = [agent_name + ".py" for agent_name in list_names]

scores = 0

df_scores = pd.DataFrame(
    scores, 
    index=list_names, 
    columns=list_names,
)

In [226]:
for i in range(len(list_names)):
    for j in range(i + 1, len(list_names)):
        print(
            f"LOG: {list_names[i]} vs {list_names[j]}", 
            end="\r"
        )
        
        current_score = evaluate(
            "rps", 
            [list_agents[i], list_agents[j]], 
            configuration={"episodeSteps": 1000}
        )
        
        df_scores[list_names[i]][list_names[j]] = current_score[0][0]
        df_scores[list_names[j]][list_names[i]] = current_score[0][1]

df_scores

,rock,paper,scissors,rand,random_r_or_p,random_s_or_p,random_s_or_r,copy_opponent,statistical
rock,0,999,-999,24,504,23,-526,0,998
paper,-999,0,999,-22,-510,519,-21,0,997
scissors,999,-999,0,0,0,-520,505,0,999
rand,-24,22,0,0,-56,0,-30,0,33
random_r_or_p,-504,510,0,56,0,304,-228,0,256
random_s_or_p,-23,-519,520,0,-304,0,282,0,37
random_s_or_r,526,21,-505,30,228,-282,0,0,446
copy_opponent,0,0,0,0,0,0,0,0,247
statistical,-998,-997,-999,-33,-256,-37,-446,-247,0
